# Lab01: Web Crawler.

- MSSV: 19120138
- Họ và tên: Trần Đức Thuỵ

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file `Lab01-WebCrawler.ipynb`, `links.txt`, `images.txt` vào, rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

**Nội dung bài tập**

Sử dụng module regex và request để thu thập đường dẫn và hình ảnh từ một trang web.

## Part 1: Thu thập dữ liệu đơn giản

### 1.1. Import

In [1]:
import requests
import re

### 1.2. Thu thập liên kết

In [2]:
r = requests.get("http://www.hcmus.edu.vn")

# TODO:
# Displays all the links present in the specified root URL - "http://www.hcmus.edu.vn"
# use regex to find all urls from r.text (http[s]://...)
# urls = ? 
r = requests.get("http://www.hcmus.edu.vn")
text = r.text
url_regex =r"http[s]*://[a-zA-Z0-9?&-=;/_]*"
urls = re.findall(url_regex, text)
urls = list(set(urls))
urls.sort(key=lambda k: len(k))
urls = [url.strip("');") for url in urls]
# Checking results?
for i in urls:
        print(i)
print(len(urls))
#Save urls to links.txt
fp = open('links.txt', 'w')
for i in urls:
    fp.write(i)
    fp.write('\n')
fp.close()

http://ogp.me/ns
http://ogp.me/ns/fb
http://www.bmrat.org/
http://ogp.me/ns/website
http://cellstemcell.org/
http://www.e-learning.vn
https://jobs.hcmus.edu.vn
https://sdh.hcmus.edu.vn/
https://www.hcmus.edu.vn/
https://hnkh.hcmus.edu.vn/
http://mail.office365.com/
http://www.hcmus.edu.vn
http://doantn.hcmus.edu.vn/
http://portal.hcmus.edu.vn/
http://hocphi.hcmus.edu.vn/
https://youtu.be/UA8eJ7NZYKc
https://www.hcmus.edu.vn/job
http://www.w3.org/1999/xhtml
https://pdt.hcmus.edu.vn/dstn
http://congdoan.hcmus.edu.vn/
https://www.hcmus.edu.vn/cbvc
https://www.hcmus.edu.vn/phone
https://research.hcmus.edu.vn/
http://tuyensinh.hcmus.edu.vn/
https://tuyensinh.hcmus.edu.vn/
https://www.facebook.com/us.vnuhcm
https://www.hcmus.edu.vn/sinh-vien
https://sdh.hcmus.edu.vn/thong-bao/
https://www.facebook.com/tvts.hcmus
https://sdh.hcmus.edu.vn/tuyen-sinh/
https://www.hcmus.edu.vn/ho-so-pgs-gs
http://www.hcmus.edu.vn/lich-cong-tac
http://ajhs.biomedpress.org/index.php/ajhs
http://stdjns.scienceandte

### 1.3. Thu thập hình ảnh

In [3]:
# TODO:
# Displays the source of all the images present in the root URL:
# img_src=?
img_src = [url for url in urls if ((".png" in url) or (".jpg" in url)) ]

# Checking results?
for i in img_src:
    print(i)

fp = open('images.txt', 'w')
for i in img_src:
    fp.write(i)
    fp.write('\n')
fp.close()

https://www.hcmus.edu.vn/images/2022/09/28/2.png
https://www.hcmus.edu.vn/images/2022/09/28/1.png
https://www.hcmus.edu.vn/images/2022/09/28/4.png
https://www.hcmus.edu.vn/images/2022/09/29/7.png
https://www.hcmus.edu.vn/images/2022/09/29/5.png
https://www.hcmus.edu.vn/images/2022/09/28/3.png
https://www.hcmus.edu.vn/images/2022/09/29/untitled-500-300-px-300-300-px-5.png
https://www.hcmus.edu.vn/images/z3336142106615_c6035997d22a0dbe3efa0a089aec7933.jpg


## Part 2. Thu thập và thể hiện dữ liệu web

### 2.1. Import library

In [4]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string

### 2.2. HTML Parser


Bộ phân tích cú pháp HTML (HTML Parser): nhận HTML code và trích xuất thông tin liên quan như tiêu đề của trang, đoạn văn trong trang, tiêu đề trong trang, liên kết, văn bản in đậm, v.v.

In [5]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)

    def handle_endtag(self, tag):
        print("End tag :", tag)

    def handle_data(self, data):
        print("Data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>')

Start tag: html
Start tag: head
Start tag: title
Data  : Test
End tag : title
End tag : head


#### Loại bỏ các HTML tag không cần thiết bằng cách thiết lập filter

In [6]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

### 2.3 Thu thập nội dung trang Web

#### Trong bài tập này ta thể hiện tài liệu (hay nội dung trang web) với cấu trúc đơn giản như sau: 
- Gọi $D$ là một tập tài liệu chứa *n* tài liệu: $D=\left\{d_1,d_2,...,d_n\right\}$.
- Ta thể hiện tài liệu bằng một dictionary `data={}` với `data[word]=[[url_idx_1,url_idx_2,...],frequency]` với `url_index`$\in{\left[{1,n}\right]}$

VD: `data['at']=[[1,2], 5]`
Từ `at` xuất hiện trong đường dẫn có index `1` và `2` tổng số lần xuất hiện là 5.

#### Bước 1: liệt kê các từ xuất hiện trong trang web:

In [7]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [8]:
def wordList(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []
    for i in range(len(filtered_text)):
        for remove_character in string.punctuation:  
            filtered_text[i] = filtered_text[i].replace(remove_character, " ")
        filtered_list = filtered_text[i].split(" ")
        word_list.extend(filtered_list)
    # TODO:
    # Với mỗi chuỗi trong filtered_text:
    #   Thay thế các dấu câu thành khoảng trắng (gợi ý: danh sách các dấu câu: string.punctuation; thay thế: .replace(...))
    #   Tách chuỗi bởi khoảng trắng (.split(...))
    #   Thêm các từ vừa được tách ra vào word_list
    word_list = [i for i in word_list if i!='']
    return word_list
  
# Test
print(wordList('https://en.wikipedia.org/wiki/Web_mining'))

['Toggle', 'sidebar', 'Search', 'Create', 'account', 'Log', 'in', 'Personal', 'tools', 'Create', 'account', 'Log', 'in', 'learn', 'more', 'Contributions', 'Talk', 'Navigation', 'Main', 'page', 'Contents', 'Current', 'events', 'Random', 'article', 'About', 'Wikipedia', 'Contact', 'us', 'Donate', 'Contribute', 'Help', 'Learn', 'to', 'edit', 'Community', 'portal', 'Recent', 'changes', 'Upload', 'file', 'Tools', 'What', 'links', 'here', 'Related', 'changes', 'Upload', 'file', 'Special', 'pages', 'Permanent', 'link', 'Page', 'information', 'Cite', 'this', 'page', 'Wikidata', 'item', 'Print', 'export', 'Download', 'as', 'PDF', 'Printable', 'version', 'In', 'other', 'projects', 'Wikimedia', 'Commons', 'Languages', 'Language', 'links', 'are', 'at', 'the', 'top', 'of', 'the', 'page', 'across', 'from', 'the', 'title', 'Contents', 'move', 'to', 'sidebar', 'hide', 'Top', '1', 'Etymology', '2', 'Background', '3', 'Process', '3', '1', 'Pre', 'processing', '3', '2', 'Data', 'mining', '3', '3', 'Resul

#### Bước 2: Tính tần suất xuất hiện của từ trong 1 trang web, lưu trữ dữ liệu vào data:

In [9]:
def read_url(url, url_idx, data):
    word_list = wordList(url)
    for word in word_list:
        if (word not in data.keys()):
            data[word] = [[url_idx], 1]
        else:
            if (url_idx not in data[word][0]):
                data[word][0].append(url_idx)
            data[word][1] = data[word][1] + 1
    # TODO
    # Với mỗi từ w trong word_list:
    #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
    #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
    

# Test
test_data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, test_data)
test_data['as']

[[1], 21]

#### Bước 3: Chạy chương trình lưu kết quả vào file output.txt

In [10]:
data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, data)
read_url('https://en.wikipedia.org/wiki/Data_mining', 2, data)

sorted_keys = sorted(data.keys())

with open("output.txt", "w", encoding="utf-8") as f:
    output_line = "Word".ljust(20) + "Frequency".ljust(15) + "URL_idx".ljust(15) + "\n"
    f.writelines(output_line)
    f.writelines('---------------------------------------------------------------------\n\n')
    for key in sorted_keys:
        output_string = str(key).ljust(20) + str(data[key][1]).ljust(15) + str(data[key][0]).ljust(15) + "\n"
        f.writelines(output_string)


In [17]:
# TODO
# Keyword tìm kiếm: python pickle
# Dùng pickle để lưu dictionary data xuống đĩa. Sau đó đọc lên và in ra 10 giá trị đầu tiên của nó.

# 1. Lưu data
import pickle

with open('output.pickle', 'wb') as f :
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# 2. Đọc lên và in ra
with open("output.pickle", "rb") as f:
    output = pickle.load(f)

print(output)

{'Toggle': [[1, 2], 4], 'sidebar': [[1, 2], 4], 'Search': [[1, 2], 2], 'Create': [[1, 2], 4], 'account': [[1, 2], 4], 'Log': [[1, 2], 4], 'in': [[1, 2], 84], 'Personal': [[1, 2], 2], 'tools': [[1, 2], 4], 'learn': [[1, 2], 2], 'more': [[1, 2], 10], 'Contributions': [[1, 2], 2], 'Talk': [[1, 2], 4], 'Navigation': [[1, 2], 2], 'Main': [[1, 2], 6], 'page': [[1, 2], 6], 'Contents': [[1, 2], 6], 'Current': [[1, 2], 2], 'events': [[1, 2], 2], 'Random': [[1, 2], 2], 'article': [[1, 2], 6], 'About': [[1, 2], 2], 'Wikipedia': [[1, 2], 4], 'Contact': [[1, 2], 2], 'us': [[1, 2], 2], 'Donate': [[1, 2], 2], 'Contribute': [[1, 2], 2], 'Help': [[1, 2], 2], 'Learn': [[1, 2], 2], 'to': [[1, 2], 98], 'edit': [[1, 2], 2], 'Community': [[1, 2], 2], 'portal': [[1, 2], 2], 'Recent': [[1, 2], 2], 'changes': [[1, 2], 4], 'Upload': [[1, 2], 4], 'file': [[1, 2], 4], 'Tools': [[1, 2], 8], 'What': [[1, 2], 2], 'links': [[1, 2], 8], 'here': [[1, 2], 4], 'Related': [[1, 2], 8], 'Special': [[1, 2], 4], 'pages': [[1,